In [164]:
from FinMind.data import DataLoader
from alpha_vantage.timeseries import TimeSeries
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from datetime import datetime, timedelta
from io import StringIO
from pprint import pprint

# question 1

## get data

In [ ]:
def get_tw_stock_by_finmind(stock, start, end):
    '''
    By FinMind API
    '''
    dl = DataLoader()
    
    stock_data = dl.taiwan_stock_daily(stock_id = stock, start_date = start, end_date = end)
    stock_data = stock_data[["Date", "open", "max", "min", "close", "Trading_Volume"]]
    stock_data = stock_data.rename(columns={
        "Date": "Date",
        "open": "Open",
        "max": "High",
        "min": "Low",
        "close": "Close",
        "Trading_Volume": "Volume"
    })

    stock_data["Date"] = pd.to_datetime(stock_data["Date"])
    stock_data.set_index("Date", inplace = True)

    return stock_data

#### FinMind 每天25次限制到了可以用 TWSE

In [238]:
def fetch_tpex_stock_data(stock_code, date):
    url = "https://www.tpex.org.tw/www/zh-tw/afterTrading/tradingStock"
    headers = {
        "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/135.0.0.0 Safari/537.36"
    }
    payload = {
        "code": stock_code,
        "date": date,
        "id": "",
        "response": "json"
    }
    res = requests.post(url, headers=headers, data=payload)
    data = json.loads(res.text)

    rows = data["tables"][0]["data"]
    columns = data["tables"][0]["fields"]

    df = pd.DataFrame(rows, columns=columns)

    df[["開盤", "最高", "最低", "收盤"]] = df[["開盤", "最高", "最低", "收盤"]].apply(pd.to_numeric, errors='coerce')
    df = df[["日 期", "開盤", "最高", "最低", "收盤", "成交仟股"]]
    df.rename(columns={
        "日 期": "Date",
        "開盤": "Open",
        "最高": "High",
        "最低": "Low",
        "收盤": "Close",
        "成交仟股": "Volume"
    }, inplace=True)
    df = df.replace(',', '', regex=True)
    df["Date"] = df["Date"].apply(convert_minguo_to_ad)
    df["Date"] = pd.to_datetime(df["Date"], format="%Y/%m/%d")

    return df

def fetch_twse_monthly_data(stock_id, year, month, fallback_to_tpex = True):
    def convert_minguo_to_ad(date_str):
        parts = date_str.split('/')
        year = int(parts[0]) + 1911
        return f"{year}/{parts[1]}/{parts[2]}"
    
    url = "https://www.twse.com.tw/exchangeReport/STOCK_DAY"
    params = {
        "response": "json",
        "date": f"{year}{month:02}01",
        "stockNo": stock_id
    }

    r = requests.get(url, params = params)
    r.encoding = 'utf-8-sig'
    data = r.json()

    if data['stat'] != 'OK':
        if fallback_to_tpex:
            date = f"{year}/{month:02}/01"
            return fetch_tpex_stock_data(stock_id, date)
        return None

    df = pd.DataFrame(data['data'], columns = data['fields'])

    df['Date'] = df['日期'].apply(convert_minguo_to_ad)
    df['Date'] = pd.to_datetime(df['Date'], format='%Y/%m/%d')
    df.drop(columns=['日期'], inplace=True)
    df = df.rename(columns={
        '日期': 'Date',
        '開盤價': 'Open',
        '最高價': 'High',
        '最低價': 'Low',
        '收盤價': 'Close',
        '成交股數': 'Volume'
    })
    df = df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
    df = df.replace(',', '', regex = True)
    df[['Open', 'High', 'Low', 'Close', 'Volume']] = df[['Open', 'High', 'Low', 'Close', 'Volume']].apply(pd.to_numeric, errors='coerce')

    return df


def get_tw_stock_by_twse(stock_id, start_date, end_date):
    start = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")

    all_data = []
    current = start.replace(day = 1)

    while current <= end:
        df = fetch_twse_monthly_data(stock_id, current.year, current.month)
        if df is not None:
            df['Date'] = pd.to_datetime(df['Date']) 
            df = df[(df['Date'] >= start) & (df['Date'] <= end)]
            all_data.append(df)
        current += timedelta(days=32)
        current = current.replace(day=1)

    result = pd.concat(all_data).sort_values('Date').set_index('Date')
    return result

In [106]:
def get_american_stock(stock, start, end):
    '''
    By Alpha Vantage API
    '''
    with open("../data/api_key.json", "r") as f:
        api_key = json.load(f)["Alpha Vantage"]["api_key"]
    
    ts = TimeSeries(key = api_key, output_format = 'pandas')
    data, _ = ts.get_daily(symbol = stock, outputsize = 'full')
    data.columns = ['Open', 'High', 'Low', 'Close', 'Volume']

    data.index = pd.to_datetime(data.index)
    data = data.sort_index(ascending = True)
    data = data.loc[start : end]

    return data

In [239]:
tickers = {
    'AMD': "2024-05-10",
    'NVDA': "2024-01-28",
    '2330.TW': "2024-03-21",
    'AAPL': "2023-12-05",
    '3324.TW': "2024-01-12"
}

total_df = pd.DataFrame()
for ticker, date in tickers.items():
    print(f"processing {ticker}...")
    
    if ticker.endswith('.TW'):
        ticker = ticker.replace('.TW', '')
        stock_data = get_tw_stock_by_twse(ticker, '2020-01-01', date)
        stock_data = stock_data["Close"]
        stock_data = stock_data.rename(ticker)
    else:
        stock_data = get_american_stock(ticker, '2020-01-01', date)
        stock_data = stock_data["Close"]
        stock_data = stock_data.rename(ticker)

    total_df = pd.concat([total_df, stock_data], axis = 1)

    time.sleep(1)

processing AMD...
processing NVDA...
processing 2330.TW...
processing AAPL...
processing 3324.TW...


In [240]:
total_df.to_csv("stock_data.csv")

## risk reward ratio

In [284]:
risk_reward_results = {}

for ticker in total_df.columns:
    series = total_df[ticker].dropna()

    net_profit = series.iloc[-1] - series.iloc[0]
    
    running_max = series.cummax()
    drawdown = (series - running_max) / running_max
    max_drawdown = drawdown.min()
    
    if max_drawdown != 0:
        risk_reward = net_profit / abs(max_drawdown)
        risk_reward_results[ticker] = {
            'Net Profit': net_profit,
            'Max Drawdown': max_drawdown,
            'Risk/Reward Ratio': risk_reward
        }
    else:
        risk_reward_results[ticker] = {
            'Net Profit': net_profit,
            'Max Drawdown': max_drawdown,
            'Risk/Reward Ratio': None
        }

risk_reward_df = pd.DataFrame(risk_reward_results).T

print(risk_reward_df)

      Net Profit  Max Drawdown  Risk/Reward Ratio
AMD       102.82     -0.654499         157.097161
NVDA      370.40     -0.864398         428.506122
2330      339.00     -0.643495         526.810704
AAPL     -106.93     -0.788891        -135.544655
3324      128.00     -0.626132         204.429752


# question2

# question3

In [285]:
latest_prices = {}

for ticker in total_df.columns:
    last_valid_idx = total_df[ticker].last_valid_index()
    latest_prices[ticker] = total_df.loc[last_valid_idx, ticker]
latest_prices = pd.Series(latest_prices)
latest_prices_df = latest_prices.reset_index()
latest_prices_df.columns = ["Ticker", "Current Price"]
latest_prices_df.drop(columns = ["Ticker"], inplace = True)

risk_reward_df.reset_index(inplace = True)
risk_reward_df = risk_reward_df["Risk/Reward Ratio"]

In [298]:
test_df = pd.read_excel("/Users/xinc./Documents/GitHub/Quant/MMA/Quant_Test_File1.xlsx")
test_df = pd.concat([test_df, risk_reward_df], axis = 1)
test_df = pd.concat([test_df, latest_prices_df], axis = 1)
test_df.rename(columns = {
    "Stock": "Stock Name",
    "Risk/Reward Ratio": "Risk Reward Ratio"
}, inplace = True)
test_df = test_df[["Stock Name", "Current Price", "Upside", "Downside", "Risk Reward Ratio"]]
test_df.to_csv("question3.csv", index = False)